# Reservation Checker Automation Project

## Part I: Introduction and remarks

This notebook is designed to automate the process of checking and managing reservations from various platforms like Booking.com, Expedia, and our website. It processes reservation data and payment reports to compare them based on different factors like if both are cancelled, if prices + refunds match between platforms etc.

Code was build on 11.3.1

Confidentiality NOTE: 
To prevent any possible information being shared files are kept at a different location and the file paths will be kept under .gitignore and only read from the variable paths but never displayed. Also in the future all seeds and some functions will be kept like that.

## Part II: Data Import and Preprocessing

In [1]:
import os
import glob
import pandas as pd

In [2]:
# Get the folder path from the hidden text file
with open('paths.txt') as f:
    paths = f.readlines()
folder_path = paths[0].strip()

In [3]:
#Import and run functions to fetch the data 
from hidden_functions import booking_api, expedia_api, mews_api

# This can be requested from the user but for the sake of example we will hard code it
start_date = '2023-04-01'
end_date = '2023-04-31'
path_to_save = folder_path

# Delete the folder if it already exists
try:
    os.rmdir(path_to_save)
except:
    pass

# Use the functions to get the data and put it in the new data in the folder 
booking_api(start_date, end_date, path_to_save)
expedia_api(start_date, end_date, path_to_save)
mews_api(start_date, end_date, path_to_save)

In [4]:
# Iterate over files in the folder
files = []
for file_path in glob.glob(folder_path):
    # Extract the file name from the file path
    file_name = file_path.split('/')[-1]
    
    # Add the file name and creation time to the list
    files.append(file_name)

    # Separate them based on file name,do already cleaning and output only useful columns
    if file_name[0:3] == 'res': #that would mean expendia 
        dt_exp = pd.read_csv(file_path)
    elif file_name[0:3] == 'Res': #that would mean our own website
        excel_full= pd.ExcelFile(file_path)
        dt_our1 = pd.read_excel(excel_full, 'Reservations')
        dt_our2 = pd.read_excel(excel_full, 'Nights')
        dt_our2.drop(['Identifier','Assigned space','Customer'],axis=1,inplace=True)
        # we do this slight juggle so that we can sum the nights per reservation but we will need the number later
        dt_our2.set_index('Number',inplace=True)
        dt_our2=pd.DataFrame(dt_our2.sum(axis=1))
        dt_our2.reset_index(inplace=True)
        dt_our1= dt_our1[['Number','Last name','First name','Status','Count (nights)','Person count','Rate','Count (bed, nightly)','Group channel manager ID','Group channel confirmation number']]
        #print(dt_our1.tail(5))
        dt_our1.drop(dt_our1.tail(1).index,inplace=True) #drop the last row as it is totals
        dt_our1['Number']=dt_our1['Number'].astype('int')
        dt_our= dt_our2.merge(dt_our1,how='right',left_on='Number',right_on='Number')
        dt_our.rename(columns={0:'Total price'},inplace=True)
    elif file_name[0:3]=="Pay": #this is a payment report, so that we can look at refunds 
        excel_full_pay = pd.ExcelFile(file_path)
        dt_pay = pd.read_excel(excel_full_pay, 'Card payments')
        dt_pay=dt_pay[['Customer','Bill','Payment card','Type','Value']]
        #drop totals row
        dt_pay.drop(dt_pay.tail(1).index,inplace=True)
        #filter for non-significant refunds
        dt_pay.drop(dt_pay[dt_pay['Value'].astype(int)>-99].index,inplace=True)
    else: #this is booking.com
        dt_bdc= pd.read_excel(file_path)
        dt_bdc['is_cancelled']=~(dt_bdc['Status'].str.contains('ok',case=False))*1
        dt_bdc.loc[:,'Prijs'] = dt_bdc['Prijs'].str.replace('EUR', '').astype('float')
        dt_bdc.loc[:,'Prijs'] = -1*dt_bdc['Prijs']* (dt_bdc['is_cancelled']-1)
        dt_bdc=dt_bdc[['Boekingsnummer','Naam gast(en)','Prijs','is_cancelled',"Status"]]
        
# Print the list of file names that were processed 
#print(files)

Split the dataset from our website into Booking.com, Expedia (and all in their affiliate network), Hotel Tonight and our own

In [5]:
#first combine names as this is what we will need to merge with the payment report 
dt_our['full_name']= dt_our['First name']+' '+dt_our['Last name']

#calculate and add BF as otherwise when we sum accross duplicates we will lose the BF
dt_our['had_bf']=(dt_our['Rate'].str.contains('Including Breakfast'))*1
dt_our['breakfast_cost']=dt_our['had_bf']*dt_our['Count (bed, nightly)']*25
dt_our['price_vat']=dt_our['Total price']+dt_our['breakfast_cost']
dt_our['tourist_fee']=dt_our['Count (bed, nightly)']*3

# Create a new column 'total_price'
dt_our[['Total price','total_price','total_breakfast']] = dt_our.groupby('full_name')[['Total price','price_vat','breakfast_cost']].transform('sum')
dt_check=dt_our[['full_name','price_vat','total_price','total_breakfast']] 
#I am aware that naming is a bit confusing but it is working, so in good coding fashion I will leave it as is

Some people have multiple reservations and to compare reliably we need to sum them and remove the duplicates 

In [6]:
# Create a new column 'Total_price_vat' to store the aggregated 'price_vat' values
dt_our['price_vat'] = dt_our.groupby('full_name')['price_vat'].transform('sum')

# Drop the duplicated entries and keep the first occurrence
dt_our.drop_duplicates(subset='Group channel confirmation number', keep='first', inplace=True)
dt_our=dt_our.merge(dt_pay,how='left',left_on='full_name',right_on='Customer')
dt_our['price_vat + refund']=dt_our['price_vat']+(dt_our['Value'].fillna(0))

## Part III: Data Analysis and Transformation

Calculate appropriate prices for our website to make them comparible with Booking and Expedia.

In [7]:
dt_our.loc[dt_our['price_vat'].isna(), 'price_vat'] = -100000000
dt_our['price_bdc']=(dt_our['price_vat']*(1.17 / 1.09)+dt_our['tourist_fee']).round(2)
dt_our.loc[dt_our['price_bdc'] < 0, ['price_vat', 'price_vat + refund', 'price_bdc','Total price']] = 0
dt_our['is_cancelled']=dt_our['Status'].str.contains('Canceled')*1
dt_our.loc[dt_our['is_cancelled']==1, ['price_vat', 'price_vat + refund', 'price_bdc']] = 0

There was a problem with the confirmation number on some of the reservations as it had some weird artefacts, so we trim them.

In [8]:
dt_our['Group channel confirmation number']=dt_our['Group channel confirmation number'].fillna(0)
dt_our['Group channel confirmation number']=dt_our['Group channel confirmation number'].astype('str')
dt_our.loc[dt_our['Group channel confirmation number'].str.len() > 10, 'Group channel confirmation number'] = dt_our['Group channel confirmation number'].str[0:9]
dt_our['Group channel manager ID']=dt_our['Group channel manager ID'].fillna('OUR')
dt_our= dt_our[['Number','Group channel confirmation number','Group channel manager ID','full_name','Status','is_cancelled','breakfast_cost','tourist_fee','Total price','price_vat + refund','Value','price_vat','price_bdc']]

Now we can split the data from our website into 4 different datasets based on where the reservation came from.

In [9]:
dt_our_bdc = dt_our[dt_our['Group channel manager ID'].str.startswith('BDC')]
dt_our_exp = dt_our[dt_our['Group channel manager ID'].str.startswith('EXP')]
dt_our_htn= dt_our[dt_our['Group channel manager ID'].str.startswith('HTN')] #not sure if i will need the last two
dt_our_our = dt_our[dt_our['Group channel manager ID'].str.startswith('OUR')]

### Booking.com

In [10]:
dt_our_bdc.loc[:,'Group channel confirmation number']=dt_our_bdc['Group channel confirmation number'].astype(float)

In [11]:
df_merged_our_bdc = dt_our_bdc.merge(dt_bdc,how='outer',left_on='Group channel confirmation number',right_on='Boekingsnummer')

Now we output a table with the reservations for which the price with the refund substracted does not match the price in Booking.

In [12]:
#df_merged_our_bdc.loc[df_merged_our_bdc['price_vat + refund']!=df_merged_our_bdc['Prijs'], ['Number','Group channel confirmation number','full_name','is_cancelled_x','is_cancelled_y','price_vat','price_vat + refund','Value',"Prijs",'breakfast_cost','tourist_fee','Boekingsnummer','Naam gast(en)']]

Reservations which are cancelled in one website but not the other.

In [13]:
#df_merged_our_bdc.loc[df_merged_our_bdc['is_cancelled_x']!=df_merged_our_bdc['is_cancelled_y'], ['Number','Group channel confirmation number','full_name','is_cancelled_x','is_cancelled_y','price_vat',"Prijs",'breakfast_cost','tourist_fee','Boekingsnummer','Naam gast(en)']]

### Expedia 

Now lets do the same for Expedia 

In [14]:
dt_exp['is_cancelled']=dt_exp['Status'].str.contains('cancelled')*1
dt_exp['Bevestigingsnr.2']=dt_exp['Bevestigingsnr.'].str[4:].astype(float)
dt_exp= dt_exp[['Gast','Boekingsbedrag','Bevestigingsnr.','Bevestigingsnr.2','is_cancelled']]
dt_our_exp.loc[:,'Group channel confirmation number']=dt_our_exp['Group channel confirmation number'].astype(float)
df_merged_our_exp = dt_our_exp.merge(dt_exp,how='outer',left_on='Group channel confirmation number',right_on='Bevestigingsnr.2')

In [15]:
#df_merged_our_exp.loc[df_merged_our_exp['price_vat + refund'].round(2)!=df_merged_our_exp['Boekingsbedrag'], ['Number','Group channel confirmation number','full_name','is_cancelled_x','is_cancelled_y','price_vat + refund','Value',"Boekingsbedrag",'breakfast_cost','tourist_fee','Gast']]

In [16]:
#df_merged_our_exp.loc[df_merged_our_exp['is_cancelled_x']!=df_merged_our_exp['is_cancelled_y'], ['Number','Group channel confirmation number','full_name','is_cancelled_x','is_cancelled_y','price_vat','Total price',"Boekingsbedrag",'breakfast_cost','tourist_fee','Gast']]

As there are not that many reservation mismatches the rest is double-checked and adjusted by hand, as it cannot be done with the API.